# **HW1: Regression** 
In *assignment 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict the number of dengue cases


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict the number of dengue cases in a different way than the basic part

# 1. Basic Part (60%)
In the first part, you need to implement the regression to predict the number of dengue cases

Please save the prediction result in a csv file **hw1_basic.csv**


## Import Packages

> Note: You **cannot** import any other package in the basic part

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

# from sklearn import linear_model
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# from sklearn.datasets import make_regression
# from sklearn.model_selection import train_test_split

## Global attributes
Define the global attributes

In [29]:
input_dataroot = 'hw1_basic_input.csv' # Input file named as 'hw1_basic_input.csv'
output_dataroot = 'hw1_basic.csv' # Output file will be named as 'hw1_basic.csv'

input_datalist =  [] # Initial datalist, saved as numpy array
output_datalist =  [] # Your prediction, should be 10 * 4 matrix and saved as numpy array
             # The format of each row should be ['epiweek', 'CityA', 'CityB', 'CityC']

You can add your own global attributes here


In [30]:

shuffle_value = 0.8

## Load the Input File
First, load the basic input file **hw1_basic_input.csv**

Input data would be stored in *input_datalist*

In [31]:
# Read input csv to datalist
with open(input_dataroot, newline='') as csvfile:
  input_datalist = np.array(list(csv.reader(csvfile)))


## Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions


### Step 1: Split Data
Split data in *input_datalist* into training dataset and validation dataset 



In [32]:

def SplitData(tmp, country_index):
  shuffled_data = np.delete(tmp, 0, 0)
  #np.random.shuffle(shuffled_data)
  rows, columns = shuffled_data.shape
  x, y = np.array(shuffled_data.T[country_index]), np.array(shuffled_data.T[country_index+3])

  str_data = np.vstack((x, y)).T
  new_data = np.asarray(str_data, dtype=float)

  training_data, vaildation_data = np.array(new_data[:int(rows*shuffle_value)]), np.array(new_data[int(rows*shuffle_value):])


  return [training_data, vaildation_data]
  #print(x_test.shape, x_vaild.shape, y_test.shape, y_vaild.shape)

### Step 2: Preprocess Data
Handle the unreasonable data
> Hint: Outlier and missing data can be handled by removing the data or adding the values with the help of statistics  

In [33]:
def PreprocessData(country_index):
    data = []
    tmp = input_datalist
    for col in range(len(input_datalist)-1, -1, -1):
        if input_datalist[col, 1] == '' or input_datalist[col, 4] == '0':
            tmp = np.delete(tmp, col, 0) #將''刪除及0刪除
    #print(tmp)
    for col in range(1, len(tmp)):
        #print(col)
        data.append(1/float(tmp[col, country_index]))
        #print(data)

    data_mean, data_std = np.mean(data), np.std(data)
    lower, upper = data_mean - data_std*1.5, data_mean + data_std*1.5
    tmp_iter = tmp
    for index in range(len(data)-1, -1, -1):
        if data[index] >= upper or data[index] <= lower:
           # print(index, tmp[index+1], data[index])
            tmp = np.delete(tmp, index+1, 0)   
            
    return tmp

  

### Step 3: Implement Regression
> Hint: You can use Matrix Inversion, or Gradient Descent to finish this part




In [34]:
def gradient(w_i, j, data, L):
    data_x, data_y = data[:, 0], data[:, 1]
    n = len(data_x)
    w_gradient_i = np.zeros(j+1)
    for i in range(n):
        x = data_x[i]
        y = data_y[i]
        x_power_list = np.power(x, np.arange(j+1)) #[1, x, x^2, ...]
        #print(x_power_list.shape, w_i.shape)
        for l in range(j+1):
            #print(l, j)
            w_gradient_i[l] = w_gradient_i[l] - (2/n)*x_power_list[l]*(y - w_i.dot(x_power_list.T))

    return w_i - w_gradient_i*L




def Regression(data, j):
    w_i = []
    data_x, data_y = data[:, 0], data[:, 1]
    #print(data_x, data_y)
    w_i = np.zeros(j+1)
    alpha = 0.000002
    repeats = 10000

    for i in range(repeats):
        w_i = gradient(w_i, j, data, alpha)

    return w_i


### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [35]:
def MakePrediction():
  

SyntaxError: unexpected EOF while parsing (1545714617.py, line 2)

### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be: 
```
3 2 1
```





In [36]:

def Mape(vaild, w_i, j):
    print(w_i)
    mape = 0.0
    N = len(vaild)
    for i in range(N):
        x_power_list = np.power(vaild[i, 0], np.arange(j+1))
        mape += (abs(vaild[i, 1] - w_i.dot(x_power_list.T)))*100/(vaild[i, 1]*N)
    return mape

    
for i in range(1, 4):
    w_i = []
    training_data, vaildation_data = SplitData(PreprocessData(i), i)
    for j in range(1, 3):
        w_i = Regression(training_data, j)
        plt.scatter(training_data[:, 0], training_data[:, 1], color ='black')
        plt.scatter(vaildation_data[:, 0], vaildation_data[:, 1], color ='green')

        x = np.arange(max(training_data[:, 0]))
        y = np.zeros(len(x))
        for k in range(len(x)):
            x_power_list = np.power(x[k], np.arange(j+1))
            y[k] = w_i.dot(x_power_list.T)
        plt.plot(x, y, color ='red') # 繪製x,y軸的圖
        mymodel = np.poly1d(np.polyfit(training_data[:, 0], training_data[:, 1], j))
        myline = np.linspace(1, max(training_data[:, 0]))
        plt.plot(myline, mymodel(myline))
        plt.show()
        mape = Mape(vaildation_data, w_i, j)
        print(mape)




KeyboardInterrupt: 

## Write the Output File
Write the prediction to output csv
> Format: 'epiweek', 'CityA', 'CityB', 'CityC'

In [ ]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow(row)

# 2. Advanced Part (35%)
In the second part, you need to implement the regression in a different way than the basic part to help your predictions for the number of dengue cases

We provide you with two files **hw1_advanced_input1.csv** and **hw1_advanced_input2.csv** that can help you in this part

Please save the prediction result in a csv file **hw1_advanced.csv** 


# Report *(5%)*

Report should be submitted as a pdf file **hw1_report.pdf**

*   Briefly describe the difficulty you encountered 
*   Summarize your work and your reflections 
*   No more than one page






# Save the Code File
Please save your code and submit it as an ipynb file! (**hw1.ipynb**)